# Pipelines

Como vimos anteriormente, quando trabalhamos com Machine Learning, o ponto mais sensível do projeto é o *dataset*. Uma vez que tudo em um projeto de Machine Learning se baseiam nele.

Para entender e interpretar o dataset podemos utilizar as seguintes etapas:

1. Coletar o máximo de informações de metadados do dataset, tais como: descritivos e domínios das features, momento e contexto da coleta, etc.;

1. Separar o dataset entre: conjunto de treino e conjunto de teste;

1. Com o conjunto de treino:

    i. Fazer uma EDA, Exploratory Data Analysis, que busque entender melhor as distribuições individuais de cada feature, assim bem como, buscar por possíveis correlações dentre elas;

    ii. Com o suporte da EDA, selecionar as features candidatas para uso na construção do modelo;

    iii. [Diagnosticar e Mitigar Anomalias](#diagnosticar-e-mitigar-anomalias):

    iv. [Normalizar as Dimensões](#normalizar-as-dimensões);

    v. Selecionar o modelo;

    vi. Treinar o modelo;

1. Com o conjunto de testes:

    i. Verificar a perfomance de acordo com as métricas adequadas.

    ii. Se necessário, voltar para o passo 3, e iterar sobre as etapas, até que se obtenha um modelo satisfatório.

Nessa seção, vamos focar nas etapas 3.iii e 3.iv, ou seja, em como diagnosticar e mitigar anomalias, e em rever como normalizar as dimensões.

<img src="https://raw.githubusercontent.com/hsandmann/biblio/refs/heads/main/ml/aula04/flow_pipeline.png" width="60%">

## Dataset

Para ilustrar a seção, vamos trabalhar com o dataset [Titanic](https://www.kaggle.com/competitions/titanic/) da Kaggle. Esse dataset é composto por dados de passageiros do navio Titanic, e o objetivo é construir um modelo que seja capaz de prever se um passageiro sobreviveu ou não ao naufrágio, a partir de suas características, como idade, sexo, classe social, etc.

A escolha desse dataset se deve ao fato de ele ser um dos mais clássicos e utilizados para fins educacionais, e por isso, já ter sido bastante explorado, o que nos permite focar nas etapas de pré-processamento, sem a necessidade de gastar muito tempo com a EDA, ou com a seleção de features.

O dataset é composto por 891 linhas, e 12 features, sendo elas:

| Feature     | Descrição                                              | Tipo       | Missing?       | Uso típico                              |
|:------------|:-------------------------------------------------------|:-----------|:--------------|:------------------------------------|
| PassengerId | ID do passageiro                                       | int        | Não           | Ignorar ou índice                   |
| Survived    | 0 = Não sobreviveu, 1 = Sobreviveu (target)            | int (0/1)  | Não           | Target                              |
| Pclass      | Classe do bilhete (1 = alta, 2 = média, 3 = baixa)     | int        | Não           | Categórica ordinal                  |
| Name        | Nome do passageiro                                     | string     | Não           | Feature engineering (títulos)       |
| Sex         | Sexo (male / female)                                   | string     | Não           | Categórica → dummy                  |
| Age         | Idade                                                  | float      | Sim (~20%)    | Imputar (média/mediana)             |
| SibSp       | Nº de irmãos/cônjuges a bordo                          | int        | Não           | Numérica                            |
| Parch       | Nº de pais/filhos a bordo                              | int        | Não           | Numérica                            |
| Ticket      | Número do bilhete                                      | string     | Não           | Pode ignorar ou extrair info        |
| Fare        | Tarifa paga                                            | float      | Poucos        | Numérica (escalar)                  |
| Cabin       | Número da cabine                                       | string     | Muitos (~77%) | Ignorar ou extrair deck             |
| Embarked    | Porto de embarque (C, Q, S)                            | string     | Poucos        | Categórica → dummy                  |


> Note que existem diversas informações faltantes, e que as features são de tipos variados, o que nos dá a oportunidade de explorar diversas técnicas de pré-processamento, e de construção de pipelines.

## Separação Treino/Teste

Esse dataset já vem separado em um conjunto de treino, e um conjunto de teste, o que é ótimo, pois nos permite focar na construção do modelo, sem a necessidade de gastar tempo com a separação dos dados.

Logo:

1. Importar as bibliotecas necessárias;
1. Carregar os datasets de treino e teste;
1. Inspecionar os datasets, para verificar se estão carregados corretamente, e para ter uma ideia geral de como eles são.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score

url_train = "https://raw.githubusercontent.com/hsandmann/biblio/refs/heads/main/ml/aula04/titanic/train.csv"

# Carregar os dados
df = pd.read_csv(url_train)

# Selecao de features e target
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = df[features]
y = df['Survived']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Inspecionando o Conjunto de Treino:

In [2]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


### Inspeção dos Targets do Conjunto de Treino

In [3]:
y_train

331    0
733    0
382    0
704    0
813    0
      ..
106    1
270    0
860    0
435    1
102    0
Name: Survived, Length: 712, dtype: int64

Como é possível notar, o dataset possui uma grande variedade de anomalias, tais como: dados faltantes, dados ruidosos, e dados inconsistentes. Além disso, as features são de tipos variados, o que nos dá a oportunidade de explorar diversas técnicas de pré-processamento, e de construção de pipelines.

## Pipelines

Note que precisamos realizar diversas etapas de [**pré-processamento**](https://scikit-learn.org/stable/modules/preprocessing.html), tais como:

- imputação de dados faltantes,
- codificação de variáveis categóricas,
- normalização de features numéricas, etc.

Para isso, podemos utilizar o conceito de pipelines, que nos permite encadear essas etapas de forma organizada e eficiente.

Ainda, quando separmos o dataset entre treino e teste, é fundamental que o processo de pré-processamento seja aplicado de forma consistente em ambos os conjuntos, para evitar vazamento de dados (*data leakage*) e para garantir que o modelo seja avaliado de forma justa.

Pipelines são uma forma de organizar o fluxo de trabalho de pré-processamento e modelagem, permitindo que as etapas sejam executadas de forma sequencial e que o código seja mais limpo e fácil de entender. Além disso, pipelines facilitam a reprodução do processo e a aplicação do mesmo processo em novos dados.

Para ilustrar o conceito de pipelines, vamos utilizar a biblioteca `sklearn`, que possui uma classe chamada [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), que nos permite encadear as etapas de pré-processamento e modelagem de forma simples e eficiente.

## Diagnosticar e Mitigar Anomalias

Todo o dataset pode apresentar problemas de faltas de dados ou dados que não tem sentido dentro do contexto da feature no dataset. Esses dados são chamados de anomalias, e podem ser classificados em três tipos:

1. Dados faltantes: quando uma feature possui valores ausentes, ou seja, não há informação disponível para aquela feature em determinada linha do dataset. Por exemplo, no dataset do Titanic, a feature `Age` possui cerca de 20% de dados faltantes.

1. Dados ruidosos: quando uma feature possui valores que são claramente errados, ou que não fazem sentido dentro do contexto da feature. Por exemplo, no dataset do Titanic, a feature "Age" possui alguns valores negativos, o que não faz sentido, pois idade não pode ser negativa.

1. Dados inconsistentes: quando uma feature possui valores que são contraditórios, ou que não seguem um padrão lógico. Por exemplo, no dataset do Titanic, a feature `Cabin` possui muitos valores faltantes, e os valores presentes são bastante variados, o que torna difícil extrair informações úteis dessa feature.

A fim de lidar com essas anomalias, podemos utilizar diversas técnicas de pré-processamento, tais como:

1. Imputação de dados faltantes: quando uma feature possui valores ausentes, podemos utilizar técnicas de imputação para preencher esses valores. Por exemplo, podemos utilizar a média ou a mediana para preencher os valores faltantes da feature "Age".

1. Remoção de dados ruidosos: quando uma feature possui valores que são claramente errados, podemos optar por remover essas linhas do dataset, ou por corrigir os valores, se possível. Por exemplo, podemos remover as linhas onde a feature `Age` possui valores negativos.

1. Tratamento de dados inconsistentes: quando uma feature possui valores que são contraditórios, ou que não seguem um padrão lógico, podemos optar por remover essa feature do dataset, ou por extrair informações úteis dela, se possível. Por exemplo, podemos optar por remover a feature `Cabin` do dataset, ou por extrair o deck da cabine, se essa informação estiver presente.

### Features Numéricas

#### Feature `Age`

Primeiro, vamos imputar dados faltantes na coluna de `Age`, a fim de poder melhor trabalhar com tal feature.

Inspecionando a coluna, podemos notar que existem valores `nan`, o que torno um desafio interpretar os dados faltantes.

**IMPUTAR**: quando precisamos substituir valores no dataset.

In [4]:
X_train['Age'].head(-5)

331    45.5
733    23.0
382    32.0
704    26.0
813     6.0
       ... 
121     NaN
614    35.0
20     35.0
700    18.0
71     16.0
Name: Age, Length: 707, dtype: float64

Para mitigar isso, podemos:

In [5]:
clean_age = df['Age'].fillna(df['Age'].median()) # aqui o pandas localizou todos os nan e trocou pela mediana
clean_age.head(-5)

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
881    33.0
882    22.0
883    28.0
884    25.0
885    39.0
Name: Age, Length: 886, dtype: float64

Agora, imagine ter que fazer isso em diversas colunas, teríamos que fazer uma função a parte apenas para realizar o pré-processamento. Para isso que existe o `Pipeline` do `sklearn`, essa classe padroniza os formatos de funções para pré-processamentos.

#### Trabalhando com todas as features numéricas

In [6]:
# Colunas numericas: imputar + escalar
numeric_features = ['Age', 'Fare', 'SibSp', 'Parch']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # mediana eh robusta para Age
])

Como vimos na aula passada, para valores numéricos, é importante uma normalização dos dados, logo, podemos deixar esse pipeline mais robusto:

In [7]:
# features numericas: imputar + escalar
numeric_features = ['Age', 'Fare', 'SibSp', 'Parch']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # mediana eh robusta para Age
    ('scaler', StandardScaler())
])

Temos aqui um pipeline na variável `numeric_tranformer` que tem duas transformações de pré-processamento em seu fluxo, sendo que:

- **a primeira**: [imputa](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) valores da mediana para dados faltantes nas colunas numéricas selecionadas. Nota: poderia ser a média também.

- **a segunda**: normaliza, [z-score](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html), as features numéricas, logo agora todas estão na mesma ordem de grandeza.

### Features Categóricas

O mesmo vale para as features categóricas, que podem apresentar dados faltantes também.

Na mesma direção, podemos utilizar o `sklearn` para mitigar esse problema:

In [8]:
# features categoricas: imputar + one-hot encoding
categorical_features = ['Sex', 'Embarked', 'Pclass']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # moda
    ('onehot', OneHotEncoder(sparse_output=False))  # converte para um vetor de zeros e uns
])

Aqui, o pipeline também cria uma variável `categorical_transformer` com duas transformações sobre os dados categóricos, sendo:

- **a primeira**: [imputa](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) valores da ocorrência de texto mais freqüente nos dados faltantes nas colunas categóricas selecionadas.

- **a segunda**: transforma as entradas em um um vetor onde cada posição representa um valor único de ocorrências, eg.:

    ``` plaintext
    Antes:
            cor tamanho  preco
    0  vermelho       P  29.90
    1      azul       M  49.90
    2     verde       G  89.90
    3  vermelho       M  45.00
    4      azul       P  32.50

    Nomes das novas colunas: ['cor_azul' 'cor_vermelho' 'tamanho_M' 'tamanho_P']

    Depois (one-hot encoding):
    preco  cor_azul  cor_vermelho  tamanho_M  tamanho_P
    0  29.90       0.0           1.0        0.0        1.0
    1  49.90       1.0           0.0        1.0        0.0
    2  89.90       0.0           0.0        0.0        0.0
    3  45.00       0.0           1.0        1.0        0.0
    4  32.50       1.0           0.0        0.0        1.0
    ```

In [9]:
# vamos tentar???

from sklearn.preprocessing import OneHotEncoder
import pandas as pd

df = pd.DataFrame({
    'cor': ['vermelho', 'azul', 'verde', 'vermelho'],
    'sexo': ['M', 'F', 'M', 'F']
})

encoder = OneHotEncoder(sparse_output=False)
X = encoder.fit_transform(df)

print(pd.DataFrame(X, columns=encoder.get_feature_names_out()))

   cor_azul  cor_verde  cor_vermelho  sexo_F  sexo_M
0       0.0        0.0           1.0     0.0     1.0
1       1.0        0.0           0.0     1.0     0.0
2       0.0        1.0           0.0     0.0     1.0
3       0.0        0.0           1.0     1.0     0.0


Pesquisando um pouco mais, existe uma outras classes: [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) e [OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html).

O que elas fazem? Quais as diferenças para o [OneHotEnconder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)? Exemplifique.

### Aplicando as Tranformações

Já vimos que é possível criar uma variável que representa uma pipeline para variáveis numéricas; assim bem como, uma outra variável que representa uma pipeline para variáveis categóricas.

Vamos combinar todas as variáveis de pipelines e colocar em uma única variável de transformação, que será responsável por todo o pré-processamento dos dados, para isso, é prático utilizar a classe [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html).

In [10]:
# comivar os pre-processadores com ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numeric_transformer, numeric_features),
        ('categorical', categorical_transformer, categorical_features)
    ]
)

---
**Pronto!!!**
---

Temos o pré-processamento definido, agora podemos acoplar o `ColumnTransforer` a um `Pipeline` e, então, aplicar sobre os dados e verificar a saída.

In [11]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

In [12]:
X_train_transformed = model.fit_transform(X_train)

# pegando os nomes das novas colunas
feature_names = preprocessor.get_feature_names_out()

# resultados
print("Shape após transformação:", X_train_transformed.shape)
print("Nomes das colunas geradas:")
print(list(feature_names))

print("Dados transformados (primeiras 6 linhas, arredondado):")
print(np.round(X_train_transformed[:6], decimals=4))

Shape após transformação: (712, 12)
Nomes das colunas geradas:
['numerical__Age', 'numerical__Fare', 'numerical__SibSp', 'numerical__Parch', 'categorical__Sex_female', 'categorical__Sex_male', 'categorical__Embarked_C', 'categorical__Embarked_Q', 'categorical__Embarked_S', 'categorical__Pclass_1', 'categorical__Pclass_2', 'categorical__Pclass_3']
Dados transformados (primeiras 6 linhas, arredondado):
[[ 1.2536 -0.0787 -0.4707 -0.4793  0.      1.      0.      0.      1.
   1.      0.      0.    ]
 [-0.4773 -0.3771 -0.4707 -0.4793  0.      1.      0.      0.      1.
   0.      1.      0.    ]
 [ 0.2151 -0.4749 -0.4707 -0.4793  0.      1.      0.      0.      1.
   0.      0.      1.    ]
 [-0.2465 -0.4762  0.3799 -0.4793  0.      1.      0.      0.      1.
   0.      0.      1.    ]
 [-1.7851 -0.0252  2.9319  2.0487  1.      0.      0.      0.      1.
   0.      0.      1.    ]
 [-0.4004  4.1387 -0.4707  0.7847  0.      1.      1.      0.      0.
   1.      0.      0.    ]]


## Concatenando Processos

Image agora o pipeline perfeito, ele faz o pré-processamento e já, na seqüência, aplica o modelo. Isso parece muito bom!

Vamos testar com nosso modelo conhecido? K-NN.

In [22]:
# ────────────────────────────────────────────────
#  Modelo: KNN com k=3
# ────────────────────────────────────────────────
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier',   KNeighborsClassifier(n_neighbors=5))
])

Lapidado, nosso pipeline está pronto, vamos aplicar em nosso dataset:

In [23]:
# treinamento
model.fit(X_train, y_train)

# previsoes
y_pred = model.predict(X_test)

# Avaliação
acc = accuracy_score(y_test, y_pred)

print("\nResultados no conjunto de teste:")
print(f"  Acurácia   = {acc:.3f}")
print("\nPredições vs Real:")
print("Predito:  ", y_pred.tolist())
print("Real:     ", y_test.tolist())


Resultados no conjunto de teste:
  Acurácia   = 0.816

Predições vs Real:
Predito:   [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1]
Real:      [1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1

Agora, todo o processo de está dentro de um pipeline, assim, é provável que erros de data leakage sejam minimizados. Também, o código é mais objeto e padronizado.

[^1]: https://scikit-learn.org/stable/data_transforms.html

[1]: Sklearn - Dataset transformations at [https://scikit-learn.org/stable/data_transforms.html](https://scikit-learn.org/stable/data_transforms.html).